<a href="https://colab.research.google.com/github/Syandilya/BasicConversions/blob/main/capstone_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

# Load the uploaded file to understand its structure
file_path = '/content/Test1.csv'
df= pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
df.head()


,id,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,3116945,8.64,x,NaN,n,t,NaN,NaN,w,11.13,...,b,NaN,w,u,w,t,g,NaN,d,a
1,3116946,6.90,o,t,o,f,NaN,c,y,1.27,...,NaN,NaN,n,NaN,NaN,f,f,NaN,d,a
2,3116947,2.00,b,g,n,f,NaN,c,n,6.18,...,NaN,NaN,n,NaN,NaN,f,f,NaN,d,s
3,3116948,3.47,x,t,n,f,s,c,n,4.98,...,NaN,NaN,w,NaN,n,t,z,NaN,d,u
4,3116949,6.17,x,h,y,f,p,NaN,y,6.73,...,NaN,NaN,y,NaN,y,t,NaN,NaN,d,u


In [4]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Copy of the dataframe to preserve the original
df_clean = df.copy()

# Step 1: Handle missing values
# For simplicity, we will fill missing numerical values with the mean and categorical values with the most frequent value
for column in df_clean.columns:
    if df_clean[column].dtype == 'object':  # Categorical column
        df_clean[column].fillna(df_clean[column].mode()[0], inplace=True)
    else:  # Numerical column
        df_clean[column].fillna(df_clean[column].mean(), inplace=True)

# Step 2: Encode categorical columns using LabelEncoder
label_encoders = {}
for column in df_clean.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df_clean[column] = le.fit_transform(df_clean[column])
    label_encoders[column] = le

# Step 3: Scale the numerical features
scaler = StandardScaler()
numerical_cols = df_clean.select_dtypes(include=['float64', 'int64']).columns
df_clean[numerical_cols] = scaler.fit_transform(df_clean[numerical_cols])

# Let's check the cleaned and encoded dataset
df_clean.head()


,id,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,-1.730419,0.400017,0.875680,0.615488,-0.302544,2.12071,-0.571112,-0.421423,0.864615,1.714916,...,-0.241123,0.02414,0.874704,0.0,0.115130,1.874805,0.356225,-0.211477,-0.508325,-0.943217
1,-1.727154,0.087888,-0.254608,0.615488,0.028042,-0.47154,-0.571112,-0.421423,1.185369,-1.821473,...,-0.241123,0.02414,-0.769628,0.0,0.115130,-0.533389,-0.315097,-0.211477,-0.508325,-0.943217
2,-1.723889,-0.791095,-1.761657,-1.368410,-0.302544,-0.47154,-0.571112,-0.421423,-0.739155,-0.060452,...,-0.241123,0.02414,-0.769628,0.0,0.115130,-0.533389,-0.315097,-0.211477,-0.508325,-0.042449
3,-1.720624,-0.527400,0.875680,0.615488,-0.302544,-0.47154,1.372593,-0.421423,-0.739155,-0.490844,...,-0.241123,0.02414,0.874704,0.0,-4.974580,1.874805,3.712836,-0.211477,-0.508325,0.858319
4,-1.717359,-0.043062,0.875680,-1.037760,1.680973,-0.47154,0.886667,-0.421423,1.185369,0.136811,...,-0.241123,0.02414,1.203570,0.0,2.659985,1.874805,-0.315097,-0.211477,-0.508325,0.858319


In [8]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, fbeta_score, mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Defining the target as 'does-bruise-or-bleed' for edibility prediction
X = df_clean.drop(columns=['does-bruise-or-bleed', 'id'])  # Feature columns
y = df_clean['does-bruise-or-bleed']  # Target column

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Building an ANN model
model = Sequential()
model.add(Dense(16, input_dim=X_train.shape[1], activation='relu'))  # Input and first hidden layer
model.add(Dense(8, activation='relu'))  # Second hidden layer
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)

# Model Evaluation
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculating metrics
#accuracy = accuracy_score(y_test, y_pred)
f2_score = fbeta_score(y_test, y_pred, beta=2)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Print the results
print("Accuracy:", accuracy)
print("F2 Score:", f2_score)
print("MSE:", mse)
print("MAE:", mae)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 


ValueError: Classification metrics can't handle a mix of continuous and binary targets

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, fbeta_score, mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Defining the target as 'does-bruise-or-bleed' for edibility prediction
X = df_clean.drop(columns=['does-bruise-or-bleed', 'id'])  # Feature columns
y = df_clean['does-bruise-or-bleed']  # Target column

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Building an ANN model
model = Sequential()
model.add(Dense(16, input_dim=X_train.shape[1], activation='relu'))  # Input and first hidden layer
model.add(Dense(8, activation='relu'))  # Second hidden layer
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)

# Model Evaluation
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Ensure y_test is binary
y_test = (y_test > 0.5).astype(int)

# Calculating metrics
accuracy = accuracy_score(y_test, y_pred) # This line was commented out
f2_score = fbeta_score(y_test, y_pred, beta=2)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Print the results
print("Accuracy:", accuracy)
print("F2 Score:", f2_score)
print("MSE:", mse)
print("MAE:", mae)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
Accuracy: 0.863849765258216
F2 Score: 0.5337078651685393
MSE: 0.13615023474178403
MAE: 0.13615023474178403


#CAT BOOST MODEL


In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, fbeta_score, mean_squared_error, mean_absolute_error

# Step 1: Handle missing values in the training dataset
df_train_clean = df_train.copy()

# Fill missing numerical values with the mean and categorical with the most frequent value
for column in df_train_clean.columns:
    if df_train_clean[column].dtype == 'object':  # Categorical column
        df_train_clean[column].fillna(df_train_clean[column].mode()[0], inplace=True)
    else:  # Numerical column
        df_train_clean[column].fillna(df_train_clean[column].mean(), inplace=True)

# Step 2: Encode categorical columns using LabelEncoder
from sklearn.preprocessing import LabelEncoder
for column in df_train_clean.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df_train_clean[column] = le.fit_transform(df_train_clean[column])

# Separate features and target for training
X_train_new = df_train_clean.drop(columns=['class', 'id'])
y_train_new = df_train_clean['class']

# Train CatBoostClassifier
catboost_model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=5, verbose=0)
catboost_model.fit(X_train_new, y_train_new)

# Test dataset (df_clean from previous cleaning)
X_test_clean = df_clean.drop(columns=['does-bruise-or-bleed', 'id'])
y_test_clean = df_clean['does-bruise-or-bleed']

# Predictions and evaluation
y_pred_catboost = catboost_model.predict(X_test_clean)

# Calculate metrics
accuracy_catboost = accuracy_score(y_test_clean, y_pred_catboost)
f2_score_catboost = fbeta_score(y_test_clean, y_pred_catboost, beta=2)
mse_catboost = mean_squared_error(y_test_clean, y_pred_catboost)
mae_catboost = mean_absolute_error(y_test_clean, y_pred_catboost)

print("Accuracy:", accuracy_catboost)
print("F2 Score:", f2_score_catboost)
print("MSE:", mse_catboost)
print("MAE:", mae_catboost)


# New Section